<a href="https://colab.research.google.com/github/shahnazumer/LCEL-DOCUMENT/blob/main/Langfuse_Integration_With_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [ ]:
%pip install langfuse langchain langchain_openai --upgrade

Initialize the Langfuse client with your API keys from the project settings in the Langfuse UI and add them to your environment.

In [ ]:
import os

# keys for project from https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # for EU data region


# openai key
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langfuse.callback import CallbackHandler

langfuse_handler = CallbackHandler()

In [ ]:
# Tests the SDK connection with the server
langfuse_handler.auth_check()

True

#Sequential Chain

In [ ]:
# further imports
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

llm = OpenAI()
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
    Title: {title}
    Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
    Play Synopsis:
    {synopsis}
    Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)
overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain],
)

# invoke
review = overall_chain.invoke("Tragedy at sunset on the beach", {"callbacks":[langfuse_handler]}) # add the handler to the run method
# run
review = overall_chain.run("Tragedy at sunset on the beach", callbacks=[langfuse_handler]) # add the handler to the run method

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


#Sequential Chain in Langchain Expression Language (LCEL)

In [ ]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

langfuse_handler = CallbackHandler()

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)
model = ChatOpenAI()
chain1 = prompt1 | model | StrOutputParser()
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"}, config={"callbacks":[langfuse_handler]})

'Chicago, Illinois se encuentra en Estados Unidos.'

#ConversationChain

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)

conversation = ConversationChain(
    llm=llm, memory=ConversationBufferMemory()
)

In [ ]:
# Create a callback handler with a session
langfuse_handler = CallbackHandler(session_id="conversation_chain")

In [ ]:
conversation.predict(input="Hi there!", callbacks=[langfuse_handler])

" Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?"

In [ ]:
conversation.predict(input="How to build great developer tools?", callbacks=[langfuse_handler])

" Building great developer tools requires a combination of technical expertise, user research, and continuous iteration. First, it's important to understand the needs and pain points of developers. This can be done through surveys, interviews, and observing their workflows. Then, the tools should be designed with a user-friendly interface and intuitive features. It's also crucial to regularly gather feedback from developers and make improvements based on their suggestions. Additionally, incorporating automation and integrations with other popular tools can greatly enhance the overall experience for developers."

In [ ]:
conversation.predict(input="Summarize your last response", callbacks=[langfuse_handler])

" To build great developer tools, it's important to understand the needs and pain points of developers through research and feedback. The tools should have a user-friendly interface, intuitive features, and incorporate automation and integrations with other popular tools. Continuous iteration and improvement based on user feedback is also crucial."

#RetrievalQA

In [ ]:
%pip install unstructured chromadb tiktoken google-search-results python-magic langchainhub --upgrade

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA

langfuse_handler = CallbackHandler()

urls = [
    "https://raw.githubusercontent.com/langfuse/langfuse-docs/main/public/state_of_the_union.txt",
]
loader = UnstructuredURLLoader(urls=urls)
llm = OpenAI()
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
query = "What did the president say about Ketanji Brown Jackson"
chain = RetrievalQA.from_chain_type(
    llm,
    retriever=docsearch.as_retriever(search_kwargs={"k": 1}),
)

chain.invoke(query, config={"callbacks":[langfuse_handler]})

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


{'query': 'What did the president say about Ketanji Brown Jackson',
 'result': ' The president said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court as a successor to retiring Justice Stephen Breyer.'}

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Determinants of LLM-assisted Decision-Making.pdf')
docs=loader.load()

### Agent

In [ ]:
from langchain.agents import AgentExecutor, load_tools, create_openai_functions_agent
from langchain_openai import ChatOpenAI
from langchain import hub

langfuse_handler = CallbackHandler()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
tools = load_tools(["serpapi"])
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "What is Langfuse?"}, config={"callbacks":[langfuse_handler]})

## Adding scores to traces

To add [scores](/docs/scores) to traces created with the Langchain integration, access the traceId via `langfuse_handler.get_trace_id()`

In [ ]:
from langfuse import Langfuse

# Trace langchain run via the Langfuse CallbackHandler as shown above

# Get id of the last created trace
trace_id = langfuse_handler.get_trace_id()

# Add score, e.g. via the Python SDK
langfuse = Langfuse()
trace = langfuse.score(
    trace_id=trace_id,
    name="user-explicit-feedback",
    value=1,
    comment="I like how personalized the response is"
)

In [15]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('/content/Determinants of LLM-assisted Decision-Making.pdf')
docs=loader.load()

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(page_content='DETERMINANTS OF LLM- ASSISTED DECISION -MAKING\nEva Eigner and\n Thorsten Händler\nFerdinand Porsche Mobile University of Applied Sciences (FERNFH)\nWiener Neustadt, Austria\neva.eigner@fernfh.ac.at; thorsten.haendler@fernfh.ac.at\nABSTRACT\nDecision-making is a fundamental capability in everyday life. Large Language Models\n(LLMs) provide multifaceted support in enhancing human decision-making processes.\nHowever, understanding the influencing factors of LLM-assisted decision-making is crucial\nfor enabling individuals to utilize LLM-provided advantages and minimize associated risks\nin order to make more informed and better decisions. This study presents the results of a\ncomprehensive literature analysis, providing a structural overview and detailed analysis of\ndeterminants impacting decision-making with LLM support. In particular, we explore the\neffects of technological aspects of LLMs, including transparency and prompt engineering,', metadata={'source': '

In [17]:
langfuse_handler = CallbackHandler()

In [18]:
## Vector Embedding And Vector Store
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings())

In [22]:
query = "Who is author of DETERMINANTS OF LLM- ASSISTED DECISION -MAKING"
retireved_results=db.similarity_search(query)

In [23]:
chain.invoke(query, config={"callbacks":[langfuse_handler]})

{'query': 'Who is author of DETERMINANTS OF LLM- ASSISTED DECISION -MAKING',
 'result': ' Eva Eigner and Thorsten Händler'}